In [ ]:
!pip install ffmpeg-python

In [ ]:
!pip install openai==0.28


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.7 MB/s eta 0:00:00


In [ ]:
!pip install gtts pydub


In [ ]:
!pip install openai==0.28 gtts pydub transformers torch tqdm SpeechRecognition


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 44.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-ma

In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from pydub import AudioSegment
from gtts import gTTS
import openai
import speech_recognition as sr
import numpy as np
import os

# Set your OpenAI API key
openai.api_key = "sk-proj-Hosg4gZzlwor4O6CJtevT3BlbkFJ6WrunCU6UjU3Uqy0LW8f"

# Load the dataset
data = pd.read_csv("/content/Medicine_Details.csv")

def preprocess_data(data):
    # Assuming all medicines are "Excellent" for simplicity
    data['Label'] = 0
    return data[['Medicine Name', 'Composition', 'Uses', 'Side_effects', 'Manufacturer', 'Label']]

data = preprocess_data(data)

class MedicineDataset(Dataset):
    def __init__(self, data, tokenizer, max_len=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        text = f"{row['Composition']} {row['Uses']} {row['Side_effects']} {row['Manufacturer']}"
        inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': torch.tensor(row['Label'], dtype=torch.long)
        }

# Split the data
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Load tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Create datasets
train_dataset = MedicineDataset(train_data, tokenizer)
test_dataset = MedicineDataset(test_data, tokenizer)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

# Load model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Set up optimizer, scheduler, and loss function
optimizer = AdamW(model.parameters(), lr=5e-5)
total_steps = len(train_loader) * 3  # 3 epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Training loop
num_epochs = 3
gradient_accumulation_steps = 2
early_stopping_patience = 2
best_loss = float('inf')
patience_counter = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, batch in enumerate(tqdm(train_loader)):
        inputs, labels = batch['input_ids'].to(device), batch['labels'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model(input_ids=inputs, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss / gradient_accumulation_steps
        loss.backward()

        if (i + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        running_loss += loss.item() * gradient_accumulation_steps

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch + 1}, Loss: {avg_loss}")

    # Early stopping
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in test_loader:
            inputs, labels = batch['input_ids'].to(device), batch['labels'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            outputs = model(input_ids=inputs, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

    val_loss /= len(test_loader)
    print(f"Validation Loss: {val_loss}")

    if val_loss < best_loss:
        best_loss = val_loss
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= early_stopping_patience:
        print("Early stopping triggered.")
        break




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

Epoch 1, Loss: 0.0175677367544169
Validation Loss: 0.00010722903897584656


100%|██████████| 592/592 [01:44<00:00,  5.66it/s]


Epoch 2, Loss: 7.078239962145351e-05
Validation Loss: 2.5554634174949496e-05


100%|██████████| 592/592 [01:44<00:00,  5.66it/s]


Epoch 3, Loss: 2.331354812975258e-05
Validation Loss: 1.0800684905818244e-05


In [ ]:
!pip install  gtts pydub transformers torch tqdm SpeechRecognition

In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from pydub import AudioSegment
from pydub.playback import play
from gtts import gTTS
import openai
import os

In [ ]:
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr






import speech_recognition as sr
from gtts import gTTS
from pydub import AudioSegment
from pydub.playback import play
import pandas as pd



# Answering questions
def answer_question(medicine_name, question, medicine_info):
    medicine_name = medicine_name.lower().strip()
    if medicine_name not in medicine_info:
        return "Medicine not found."

    info = medicine_info[medicine_name]
    if "use" in question.lower():
        return info.get('Uses', 'No information available.')
    elif "side effect" in question.lower():
        return info.get('Side_effects', 'No information available.')
    else:
        return "I can only answer questions about uses and side effects."

# Convert text to speech and save to file
def text_to_speech(text, filename="response.mp3"):
    tts = gTTS(text=text, lang='en')
    tts.save(filename)
    return filename




# Play audio function
def play_audio(audio_data, sample_rate):
    display(Audio(data=audio_data, rate=sample_rate))




def speech_to_text(audio_data, sample_rate):
    recognizer = sr.Recognizer()
    audio_data = audio_data.astype('int16').tobytes()
    audio_data_obj = sr.AudioData(audio_data, sample_rate=sample_rate, sample_width=2)

    try:
        text = recognizer.recognize_google(audio_data_obj,  language='en-US', show_all=False)
        return text
    except sr.UnknownValueError:
        return "Sorry, I could not understand the audio."
    except sr.RequestError:
        return "Sorry, my speech recognition service is currently unavailable."

sample_rate = 48000
# Example usage
if __name__ == "__main__":
    # Convert all values in the 'Medicine Name' column to lowercase
    data['Medicine Name'] = data['Medicine Name'].str.lower()

    # Convert the data to a dictionary
    medicine_info = data.set_index('Medicine Name').T.to_dict()

    # Record user's question
    medicine_name_aud, sr1 =  get_audio()
    question_text_aud, sr2= get_audio()

    # Get the answer from the chatbot
    medicine_name = speech_to_text(medicine_name_aud,  sample_rate=sample_rate)  # Replace with appropriate medicine name
    question_text = speech_to_text(question_text_aud, sample_rate=sample_rate)
    print (f"Medicine: {medicine_name}")
    print(f"User Question: {question_text}")
    answer_text = answer_question(medicine_name, question_text, medicine_info)
    print(f"Chatbot Answer: {answer_text}")

    # Convert the chatbot answer to speech
    tts_output_file = text_to_speech(answer_text)
    print(f"Audio response saved to {tts_output_file}.")

    # Play the audio response
    play_audio(tts_output_file, sample_rate=sample_rate)









<ipython-input-48-80c2391fe96e>:178: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  medicine_info = data.set_index('Medicine Name').T.to_dict()


Medicine: azithral 500 tablet
User Question: uses
Chatbot Answer: Treatment of Bacterial infections
Audio response saved to response.mp3.


In [ ]:
data.head()

,Medicine Name,Composition,Uses,Side_effects,Manufacturer,Label
0,Avastin 400mg Injection,Bevacizumab (400mg),Cancer of colon and rectum Non-small cell lun...,Rectal bleeding Taste change Headache Noseblee...,Roche Products India Pvt Ltd,0
1,Augmentin 625 Duo Tablet,Amoxycillin (500mg) + Clavulanic Acid (125mg),Treatment of Bacterial infections,Vomiting Nausea Diarrhea Mucocutaneous candidi...,Glaxo SmithKline Pharmaceuticals Ltd,0
2,Azithral 500 Tablet,Azithromycin (500mg),Treatment of Bacterial infections,Nausea Abdominal pain Diarrhea,Alembic Pharmaceuticals Ltd,0
3,Ascoril LS Syrup,Ambroxol (30mg/5ml) + Levosalbutamol (1mg/5ml)...,Treatment of Cough with mucus,Nausea Vomiting Diarrhea Upset stomach Stomach...,Glenmark Pharmaceuticals Ltd,0
4,Aciloc 150 Tablet,Ranitidine (150mg),Treatment of Gastroesophageal reflux disease (...,Headache Diarrhea Gastrointestinal disturbance,Cadila Pharmaceuticals Ltd,0
